In [1]:
import traitlets
from jetcam.utils import bgr8_to_jpeg
from IPython.display import display
import ipywidgets
from jetcam.csi_camera import CSICamera
import cv2
import torch
from torch import nn, optim
import numpy as np
import random
import sys
from src.vgg_local import VGG_LOCAL
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import gc

In [2]:
model_path = "final_weight.pth"
model_ver = "VGG11"

38

In [7]:
gc.collect()
# カメラデバイスの初期化
camera = CSICamera(width=64, height=64, capture_device=0)  # カメラの解像度を適宜設定

# 画像のキャプチャ
image = camera.read()

gc.collect()
del camera

# 画像を保存
image_path = "image.jpg"
cv2.imwrite(image_path, image)  # OpenCVを使用して画像を保存



(python3:12348): GStreamer-WARNING **: 06:06:00.912: Failed to load plugin '/usr/lib/aarch64-linux-gnu/gstreamer-1.0/libgstnvarguscamerasrc.so': /usr/lib/aarch64-linux-gnu/libGLdispatch.so.0: cannot allocate memory in static TLS block

(python3:12348): GStreamer-WARNING **: 06:06:00.916: Failed to load plugin '/usr/lib/aarch64-linux-gnu/gstreamer-1.0/libgstnvvidconv.so': /usr/lib/aarch64-linux-gnu/libGLdispatch.so.0: cannot allocate memory in static TLS block
[ WARN:0@189.815] global cap_gstreamer.cpp:1697 open OpenCV | GStreamer warning: unable to query duration of stream

(python3:12348): GStreamer-CRITICAL **: 06:06:30.924: gst_caps_get_structure: assertion 'GST_IS_CAPS (caps)' failed

(python3:12348): GStreamer-CRITICAL **: 06:06:30.924: gst_structure_get_int: assertion 'structure != NULL' failed
[ WARN:0@189.815] global cap_gstreamer.cpp:1707 open OpenCV | GStreamer warning: cannot query video width/height

(python3:12348): GStreamer-CRITICAL **: 06:06:30.924: gst_structure_get_f

In [4]:
image_size = 64
# Set random seed for reproducibility
random_seed = 9999
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
else:
    device = torch.device('cpu')
# Preprocessing for test data
test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# input = test_transform(Image.fromarray(image))
# Initialize and load model weights
model = VGG_LOCAL(model_ver, classes=3, image_size=image_size).to(device)
model.load_state_dict(torch.load('../'+model_path))
model.eval()
with torch.no_grad():
    input = input.to(device)
    output = model(input)